<a href="https://colab.research.google.com/github/anjanadevi05/Defect_Detection/blob/main/defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Defect detection using deeplearning models***

In [ ]:
import os
from os import path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importing required libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img ,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix

# DATA LOADING

In [ ]:
import zipfile
import os

# Assume your file is named 'your_file.zip'
filename = '/content/casting_data.zip'

# Unzip the file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('extracted_folder')  # Specify the folder where you want to extract the files

# List the extracted files
os.listdir('extracted_folder')


['casting_data']

In [ ]:
import os

# Define the base directory after extracting your zip file
my_data_dir = '/content/extracted_folder/casting_data/'  # Adjust according to your actual path

# Define train and test paths
train_path = os.path.join(my_data_dir, 'train')
test_path = os.path.join(my_data_dir, 'test')

# Parameters for image processing
image_shape = (300, 300, 1)
batch_size = 32

# Verify the structure by listing contents of the directories
print("Train directory contents:", os.listdir(train_path))
print("Test directory contents:", os.listdir(test_path))


Train directory contents: ['def_front', 'ok_front']
Test directory contents: ['def_front', 'ok_front']


# DATA PREPROCESSING

In [ ]:
image_gen = ImageDataGenerator(rotation_range=75,
                               width_shift_range=0,
                               height_shift_range=0,
                               rescale=1/255,
                               shear_range=0,
                               zoom_range=0,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )

In [ ]:
train_set = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode="grayscale",
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=True)

test_set = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode="grayscale",
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


In [ ]:
model = Sequential()

In [ ]:
image_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_set.class_indices

{'def_front': 0, 'ok_front': 1}

# CNN


In [ ]:
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=16, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=16, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(224))
model.add(Activation('relu'))
# Last layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
results = model.fit(train_set,epochs=20, # Use fit instead of fit_generator
                              validation_data=test_set,
                             callbacks=[early_stop])

Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.6539 - loss: 0.6640 - val_accuracy: 0.8336 - val_loss: 0.3340
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9224 - loss: 0.2141 - val_accuracy: 0.8727 - val_loss: 0.2650
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.9671 - loss: 0.0923 - val_accuracy: 0.9846 - val_loss: 0.0597
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9752 - loss: 0.0754 - val_accuracy: 0.9944 - val_loss: 0.0273
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9834 - loss: 0.0522 - val_accuracy: 0.9902 - val_loss: 0.0216
Epoch 6/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9919 - loss: 0.0319 - val_accuracy: 0.9902 - val_loss: 0.0211
Epoch 7/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9964 - loss: 0.0156 - val_accuracy: 0.9944 - val_loss: 0.0151
Epoch 8/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 314s 2s/step - accuracy: 0.9947 - loss: 0.0189 - val_accu

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_set)

print(f"Test Accuracy: {accuracy * 100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 516ms/step - accuracy: 0.9966 - loss: 0.0182
Test Accuracy: 99.44%


# ResNet


In [ ]:
!pip install transformers torch


In [ ]:
from transformers import ResNetForImageClassification, AutoImageProcessor
from tensorflow.keras.optimizers import Adam
from transformers import AdamW
from torchvision import transforms
import torch
import numpy as np
from PIL import Image
import tensorflow as tf

In [ ]:
# Load the ResNet model from Hugging Face and the processor for preprocessing
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50", num_labels=2, ignore_mismatched_sizes = True)
# Binary classification # Added ignore_mismatched_sizes = True to handle the mismatch in the number of classes in the classifier layer

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ResNetDataGenerator(Dataset):
    def __init__(self, image_directory, processor):
        self.image_directory = image_directory
        self.processor = processor
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        # Use the processor to preprocess the image
        image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB format
        inputs = self.processor(images=image, return_tensors="pt")  # Use processor for preprocessing

        return inputs['pixel_values'].squeeze(0), torch.tensor(label)  # Return image and label


In [ ]:
from torch.utils.data import DataLoader

train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

# Create instances of the data generator
train_data_gen = ResNetDataGenerator(train_directory, processor)
test_data_gen = ResNetDataGenerator(test_directory, processor)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)


In [ ]:
print(train_data_gen.class_to_idx)
print(test_data_gen.class_to_idx)

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=3, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = None
        self.counter = 0

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                return True  # Stop training
        else:
            self.best_loss = val_loss
            self.counter = 0
        return False  # Continue training

# Create an instance of EarlyStopping
early_stopping = EarlyStopping(patience=3, verbose=True)

# Training loop
num_epochs = 5  # Set the number of epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)  # Move to GPU if available
        optimizer.zero_grad()

        outputs = model(images).logits  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Calculate validation loss for early stopping
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(test_loader)  # Average validation loss
    print(f'Validation Loss: {val_loss:.4f}')

    # Check for early stopping
    if early_stopping(val_loss):
        print("Early stopping triggered")
        break  # Stop training


100%|██████████| 208/208 [01:36<00:00,  2.15it/s]


Epoch [1/5], Loss: 0.0048
Validation Loss: 0.0113


100%|██████████| 208/208 [01:34<00:00,  2.20it/s]


Epoch [2/5], Loss: 0.0049
Validation Loss: 0.0055


100%|██████████| 208/208 [01:35<00:00,  2.19it/s]


Epoch [3/5], Loss: 0.0037
Validation Loss: 0.0085
EarlyStopping counter: 1 out of 3


100%|██████████| 208/208 [01:34<00:00,  2.19it/s]


Epoch [4/5], Loss: 0.0019
Validation Loss: 0.0083
EarlyStopping counter: 2 out of 3


100%|██████████| 208/208 [01:35<00:00,  2.18it/s]


Epoch [5/5], Loss: 0.0019
Validation Loss: 0.0046


In [ ]:
# Set the model to evaluation mode
model.eval()

# Initialize variables to track the correct predictions and total samples
correct_predictions = 0
total_samples = 0

# Disable gradient calculation for efficiency
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move to GPU if available

        outputs = model(images).logits  # Get model outputs
        _, predicted = torch.max(outputs, 1)  # Get predicted labels
        total_samples += labels.size(0)  # Update total samples
        correct_predictions += (predicted == labels).sum().item()  # Count correct predictions

# Calculate test accuracy
test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Test Accuracy: 99.72%


# Xception

In [ ]:
!pip install timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.2 MB/s eta 0:00:00


In [ ]:
import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [ ]:
# Load the Xception model from timm FOR BINARY CLASSIFICATION
model = timm.create_model('xception', pretrained=True, num_classes=2)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [ ]:
# Define a data generator class
class XceptionDataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        # Open image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations (resize, normalize, etc.)
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [ ]:
# Define transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Xception expects 299x299 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Set up dataset and dataloader
train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

train_data_gen = XceptionDataGenerator(train_directory, transform=transform)
test_data_gen = XceptionDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)


In [ ]:
import timm
import torch.nn as nn

# Load the Xception model from timm
model = timm.create_model('xception', pretrained=True, num_classes=2)

# Check the model structure
print(model)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Ensure that model is not a Sequential
criterion = nn.CrossEntropyLoss()


Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): ReLU(inplace=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv

In [ ]:
from tqdm import tqdm  # Import tqdm for progress bars

# Parameters for early stopping
patience = 3  # Number of epochs to wait for improvement
best_loss = float('inf')  # Initialize the best loss
patience_counter = 0  # Counter for patience

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Use tqdm for progress bar
    for images, labels in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Validation step
    model.eval()
    total_val_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for val_images, val_labels in tqdm(test_loader, desc='Validating'):
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            total_val_loss += val_loss.item()

            _, predicted = torch.max(val_outputs, 1)
            total_samples += val_labels.size(0)
            correct_predictions += (predicted == val_labels).sum().item()

    avg_val_loss = total_val_loss / len(test_loader)
    val_accuracy = correct_predictions / total_samples

    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%')

    # Early stopping logic
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience_counter = 0  # Reset the counter
        print("Model improved, saving the model...")
        # Optionally save the model here
    else:
        patience_counter += 1
        print(f'EarlyStopping counter: {patience_counter} out of {patience}')

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


Training Epoch 1/5: 100%|██████████| 208/208 [03:50<00:00,  1.11s/it]


Epoch [1/5], Loss: 0.0108


Validating: 100%|██████████| 23/23 [00:09<00:00,  2.53it/s]


Validation Loss: 0.0065, Validation Accuracy: 99.72%
Model improved, saving the model...


Training Epoch 2/5: 100%|██████████| 208/208 [03:48<00:00,  1.10s/it]


...

Validation Loss: 0.0072, Validation Accuracy: 99.72%
EarlyStopping counter: 2 out of 3


Training Epoch 4/5: 100%|██████████| 208/208 [03:47<00:00,  1.10s/it]


Epoch [4/5], Loss: 0.0041


Validating: 100%|██████████| 23/23 [00:08<00:00,  2.56it/s]

Validation Loss: 0.0131, Validation Accuracy: 99.58%
EarlyStopping counter: 3 out of 3
Early stopping triggered.


In [ ]:
# Testing loop
model.eval()  # Set the model to evaluation mode
total_test_loss = 0
correct_predictions = 0
total_samples = 0

with torch.no_grad():  # Disable gradient calculation for testing
    for test_images, test_labels in tqdm(test_loader, desc='Testing'):
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        # Get model outputs
        test_outputs = model(test_images)

        # Calculate loss
        test_loss = criterion(test_outputs, test_labels)
        total_test_loss += test_loss.item()

        # Get predicted classes
        _, predicted = torch.max(test_outputs, 1)
        total_samples += test_labels.size(0)
        correct_predictions += (predicted == test_labels).sum().item()

# Calculate average test loss and accuracy
avg_test_loss = total_test_loss / len(test_loader)
test_accuracy = correct_predictions / total_samples

print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')


Testing: 100%|██████████| 23/23 [00:09<00:00,  2.49it/s]

Test Loss: 0.0131, Test Accuracy: 99.58%


# GoogLeNet

In [ ]:
import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os

In [ ]:
# Load GoogLeNet model for Binary classification
model = models.googlenet(pretrained=True)

# Modify the final fully connected layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:01<00:00, 26.3MB/s]


In [ ]:
# Define a data generator class
class GoogleNetDataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        # Open image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations (resize, normalize, etc.)
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # GoogLeNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Set up dataset and dataloader
train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

train_data_gen = XceptionDataGenerator(train_directory, transform=transform)
test_data_gen = XceptionDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
import torch

class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered.")

# Initialize early stopping
early_stopping = EarlyStopping(patience=3, verbose=True)

# Training loop with early stopping
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    # Training
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_correct / total_samples * 100

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.2f}%')

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    val_correct = 0
    val_samples = 0

    with torch.no_grad():  # Disable gradient calculation for validation
        for val_images, val_labels in test_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == val_labels).sum().item()
            val_samples += val_labels.size(0)

    avg_val_loss = val_loss / len(test_loader)
    avg_val_accuracy = val_correct / val_samples * 100

    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.2f}%')

    # Check for early stopping
    early_stopping(avg_val_loss)

    if early_stopping.early_stop:
        print("Early stopping invoked. Training has stopped.")
        break


Epoch 1/20: 100%|██████████| 208/208 [00:46<00:00,  4.46it/s]


Epoch [1/20], Loss: 0.0052, Accuracy: 99.85%
Validation Loss: 0.0076, Validation Accuracy: 99.72%


Epoch 2/20: 100%|██████████| 208/208 [00:44<00:00,  4.63it/s]


Epoch [2/20], Loss: 0.0023, Accuracy: 99.95%
Validation Loss: 0.0106, Validation Accuracy: 99.86%


Epoch 3/20: 100%|██████████| 208/208 [00:48<00:00,  4.30it/s]


...

Epoch [4/20], Loss: 0.0038, Accuracy: 99.88%
Validation Loss: 0.0096, Validation Accuracy: 99.72%


Epoch 5/20: 100%|██████████| 208/208 [00:44<00:00,  4.65it/s]


Epoch [5/20], Loss: 0.0017, Accuracy: 99.97%
Validation Loss: 0.0093, Validation Accuracy: 99.72%


Epoch 6/20: 100%|██████████| 208/208 [00:44<00:00,  4.70it/s]


Epoch [6/20], Loss: 0.0005, Accuracy: 100.00%
Validation Loss: 0.0104, Validation Accuracy: 99.72%
Early stopping triggered.
Early stopping invoked. Training has stopped.


In [ ]:
# Validation loop
model.eval()  # Set model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():  # Disable gradient calculation for validation
    # Use tqdm to show progress for the validation loop
    for images, labels in tqdm(test_loader, desc='Validation'):
        images, labels = images.to(device), labels.to(device)  # Move to device
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        total_samples += labels.size(0)  # Update total samples
        correct_predictions += (predicted == labels).sum().item()  # Update correct predictions

# Calculate test accuracy
test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Validation: 100%|██████████| 23/23 [00:02<00:00,  7.71it/s]

Test Accuracy: 99.72%


#InceptionV3

In [ ]:
import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os

In [ ]:
# Load InceptionV3 model for binary classification
model = timm.create_model('inception_v3', pretrained=True, num_classes=2)
model = model.to(device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Set up dataset and dataloader (same as before)
train_data_gen = XceptionDataGenerator(train_directory, transform=transform)
test_data_gen = XceptionDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)


In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Define a data generator class
class InceptionV3_DataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        # Open image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations (resize, normalize, etc.)
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [ ]:
# Parameters for early stopping
patience = 3  # Number of epochs to wait for improvement
best_loss = float('inf')  # Initialize the best loss
patience_counter = 0  # Counter for patience

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Use tqdm for progress bar
    for images, labels in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Validation step
    model.eval()
    total_val_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for val_images, val_labels in tqdm(test_loader, desc='Validating'):
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            total_val_loss += val_loss.item()

            _, predicted = torch.max(val_outputs, 1)
            total_samples += val_labels.size(0)
            correct_predictions += (predicted == val_labels).sum().item()

    avg_val_loss = total_val_loss / len(test_loader)
    val_accuracy = correct_predictions / total_samples

    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%')

    # Early stopping logic
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience_counter = 0  # Reset the counter
        print("Model improved, saving the model...")
        # Optionally save the model here
    else:
        patience_counter += 1
        print(f'EarlyStopping counter: {patience_counter} out of {patience}')

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

Training Epoch 1/5: 100%|██████████| 208/208 [02:04<00:00,  1.66it/s]


Epoch [1/5], Loss: 0.0097


Validating: 100%|██████████| 23/23 [00:06<00:00,  3.72it/s]


Validation Loss: 0.0118, Validation Accuracy: 99.72%
Model improved, saving the model...


Training Epoch 2/5: 100%|██████████| 208/208 [02:01<00:00,  1.71it/s]


...

Validation Loss: 0.0098, Validation Accuracy: 99.58%
Model improved, saving the model...


Training Epoch 5/5: 100%|██████████| 208/208 [02:01<00:00,  1.71it/s]


Epoch [5/5], Loss: 0.0019


Validating: 100%|██████████| 23/23 [00:06<00:00,  3.58it/s]

Validation Loss: 0.0047, Validation Accuracy: 99.58%
Model improved, saving the model...


In [ ]:
# Testing loop
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    # Use tqdm for progress bar during testing
    for images, labels in tqdm(test_loader, desc='Testing'):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predicted classes
        _, predicted = torch.max(outputs, 1)

        # Update total samples and correct predictions
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

# Calculate accuracy
test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Testing: 100%|██████████| 23/23 [00:06<00:00,  3.61it/s]

Test Accuracy: 99.58%


 AlexNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os
from tqdm import tqdm

# Dataset Class
class CustomDataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Directories
train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

train_data_gen = CustomDataGenerator(train_directory, transform=transform)
test_data_gen = CustomDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)

# Load AlexNet Pretrained Model
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)  # Adjust for 2 classes

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Test Accuracy
model.eval()
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


100%|██████████| 208/208 [00:30<00:00,  6.83it/s]


Epoch [1/20], Loss: 0.5776


100%|██████████| 208/208 [00:29<00:00,  7.16it/s]


Epoch [2/20], Loss: 0.4229


100%|██████████| 208/208 [00:29<00:00,  7.11it/s]


...

Epoch [18/20], Loss: 0.0386


100%|██████████| 208/208 [00:30<00:00,  6.93it/s]


Epoch [19/20], Loss: 0.0302


100%|██████████| 208/208 [00:29<00:00,  7.09it/s]


Epoch [20/20], Loss: 0.0546
Test Accuracy: 99.72%


VGG16

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os
from tqdm import tqdm

# Dataset Class (Same as in AlexNet)
class CustomDataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

# Image Transformations (Same as in AlexNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Directories
train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

train_data_gen = CustomDataGenerator(train_directory, transform=transform)
test_data_gen = CustomDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)

# Load VGG16 Pretrained Model
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 2)  # Adjust for 2 classes

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Test Accuracy
model.eval()
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


100%|██████████| 208/208 [01:53<00:00,  1.83it/s]


Epoch [1/10], Loss: 0.5924


100%|██████████| 208/208 [01:52<00:00,  1.84it/s]


Epoch [2/10], Loss: 0.3299


100%|██████████| 208/208 [01:53<00:00,  1.84it/s]


...

100%|██████████| 208/208 [01:52<00:00,  1.84it/s]


Epoch [6/10], Loss: 0.0587


100%|██████████| 208/208 [01:52<00:00,  1.85it/s]


Epoch [7/10], Loss: 0.0467


 15%|█▌        | 32/208 [00:17<01:34,  1.85it/s]

 DenseNet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import os
from tqdm import tqdm

# Dataset Class (Same as in AlexNet)
class CustomDataGenerator(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, sub_dir in enumerate(['def_front', 'ok_front']):
            sub_dir_path = os.path.join(self.image_directory, sub_dir)
            if os.path.exists(sub_dir_path):
                for file in os.listdir(sub_dir_path):
                    file_path = os.path.join(sub_dir_path, file)
                    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                        self.image_files.append(file_path)
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

# Image Transformations (Same as in AlexNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Directories
train_directory = '/content/extracted_folder/casting_data/train'
test_directory = '/content/extracted_folder/casting_data/test'

train_data_gen = CustomDataGenerator(train_directory, transform=transform)
test_data_gen = CustomDataGenerator(test_directory, transform=transform)

train_loader = DataLoader(train_data_gen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_gen, batch_size=32, shuffle=False)

# Load DenseNet Pretrained Model
model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(1024, 2)  # Adjust for 2 classes

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Loss
optimizer = optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Test Accuracy
model.eval()
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

test_accuracy = correct_predictions / total_samples
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 110MB/s]
100%|██████████| 208/208 [01:22<00:00,  2.51it/s]


Epoch [1/5], Loss: 0.0500


100%|██████████| 208/208 [01:23<00:00,  2.49it/s]


Epoch [2/5], Loss: 0.0191


100%|██████████| 208/208 [01:25<00:00,  2.44it/s]


Epoch [3/5], Loss: 0.0199


100%|██████████| 208/208 [01:24<00:00,  2.46it/s]


Epoch [4/5], Loss: 0.0118


100%|██████████| 208/208 [01:24<00:00,  2.47it/s]


Epoch [5/5], Loss: 0.0160
Test Accuracy: 99.72%
